In [2]:
# recurisvely defined fibs
# most intuitive implementation is recursive
def fib(n):
    if n in [0, 1]:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


# works ok
# but to calculate fib(10) algorithm has to calculate fib(9) and fib(8) first.
map(fib, range(10))


In [3]:
# using memoization instead
def fib(n):
    # iterative implementation, memoized
	x, y = 0, 1
	for _ in range(n):
		x, y = y, x + y
	return y

- Haskell embraces laziness where values are only computed as needed, meaning you can compute the infinite sequence of Fibonacci numbers.
```Haskell
	fibs :: [Int]
	fibs = 1 : 1 : zipWith (+) fibs (tail fibs)

```
- in ghci write 'take 10 fibs' to see the first 10 numbers in the sequence

- The code/definition above should be understood as: "the sequence that starts with 1, followed by 1, followed by the sequence of numbers obtained by "zipping" together the sequences `fibs` and `tail fibs` (that is, all of the elements of `fibs` after the first) and adding together each corresponding pair of elements."
- Which means the third element of the sequence is the first element (of `fibs`) plus the second element (which is the first element of `tail fibs`). The fourth element is the second element of `fibs` plus the third element ( which is the second element of `tail fibs`), and so on. It only sounds circular because we're using `fibs` to define `fibs`.
- the following sort of thing leads to infinite recursion:

In [ ]:
# python implementation of the haskell 'style' `fibs`
def fibs(x = 0, y = 1):
    return [y] + fibs(y, x + y)

# print(fibs()[1])
# results in kernel crashing.
# doesnt even return the proper RuntimeError message

: 

: 

In [6]:
# Laziness in Python using Generators
num = (i for i in range(3))
num

<generator object <genexpr> at 0x000002393D977BC0>

In [8]:
num.next()
# 3.10.8 returns AttributeError: 'generator' object has no attribute 'next'

AttributeError: 'generator' object has no attribute 'next'

In [9]:
def to3():
    yield 1
    yield 2
    yield 3


[x for x in to3()]

[1, 2, 3]

- Haskell implementation used `tail` (to get the elements after the first) and `take` (to get a certain number of elements from the front). Python doesnt have these (yet?), so we'll use `islice` (which allows you to slice a new generator out of an old one) from the `itertools` module:

In [10]:
from itertools import islice

def tail(iterable):
	"""Returns all but the first element of an iterable."""
	return islice(iterable, 1, None)

def take(n, iterable):
	"""Returns first n items of the iterable as a list"""
	return list(islice(iterable, 0, n))

In [11]:
def fibs():
	x, y = (0, 1)
	while True:
		yield y
		(x, y) = (y, x + y)

def boring_fibs():
    cur = next = 1
	while True:
		yield cur
		cur, next = next, cur + next

In [12]:
take(10, fibs())

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

In [13]:
from operator import add
from itertools import tee

def fibs():
	yield 1
	yield 1
	yield from map(add, fibs(), tail(fibs()))

# or as a simple one-liner
def fibs():
	print(f"a new fibs")
	yield 1
	yield 1
	fibs1, fibs2 = tee(fibs())
	yield from map(add, fibs1, tail(fibs2()))


In [14]:
# using lambdas
square = lambda x: x**2
some_list = [(1, 3), (5, 2), (6, 1), (4, 6)]
some_list.sort(key = lambda x: x[0]**2 - x[1]*3)
some_list

[(1, 3), (4, 6), (5, 2), (6, 1)]

In [16]:
function_product = lambda F, m: lambda x: F(x)*m
print(square(2))

function_product(square, 3)(2)

4


12

In [18]:
fibonacho = (lambda x, x_1 = 1, x_2 = 0: x_2 if x == 0 else fibonacho(x - 1, x_1 + x_2, x_1))
print(fibonacho(1))
print(fibonacho(5))
print(fibonacho(6))

1
5
8


In [24]:
display(map(lambda x: x**2, [1, 2, 3]))
filter(lambda x: x % 2 == 0, [1, 2, 3, 4, 5])


In [26]:
# yairchu.github.io/posts/leet-haskell-in-python
import itertools

def leet(gen):
	"""
	decorate a fx returning a generator
	to memoize its consumed values for all eternity, amen.
	"""
	original = gen()
	as_list = []
	def result():
		for i in itertools.count():
			if i == len(as_list):
				as_list.append(next(original))
			yield as_list[i]
	return result
			

@leet  # essential decorator
def fibs():
	yield 1; yield 1
	yield from map(add, fibs(), islice(fibs(), 1, None))

In [ ]:
# different implementation of lazy fibs in python
def lazy_quicky(gen):
	"""
	Haskell-style lazy quicksort
	Doesnt need to full sort everything to find the n smallest elements.

	Complexity would be O(n log k) where k is the amount iterated, in comparison
	to O(n log n) for the full sorting algorithm.

	Canonical example is iterating over potential dating candidates
	ordered by their level of attractiveness.
	Until finding the one girl that will agree to go out with you.
	"""
	gen = iter(gen)
	try:
		pivot = next(gen)
	except StopIteration:
		return
	less = [], more = []
	for x in gen:
		(less if x < pivot else more).append(x)
	yield from lazy_quicky(less)
	yield pivot
	yield from lazy_quicky(more)
	